In [1]:
from Utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import os.path

#import warnings
#warnings.filterwarnings("ignore")
#warnings.filterwarnings(action="ignore",category=DeprecationWarning)
#warnings.filterwarnings(action="ignore",category=FutureWarning)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import gc
import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


from keras.layers import Dense, Input, Activation
from keras.layers import BatchNormalization,Add,Dropout
from keras.optimizers import Adam, RMSprop
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K


Using TensorFlow backend.


In [38]:
coupling_type = '2JHN'
train_csv = load_train()
n_atoms = 7
XY_Data = build_XY(train_csv, coupling_type, n_atoms, False)
#XY_Data = pd.read_csv(f'{Config.INPUT_XY}/{coupling_type}.csv', index_col=0)
XY_Data, _ = train_test_split(XY_Data, test_size=0.9, random_state=42)

In [39]:
#X_Data, y_Data = build_x_y_data(XY_Data)    
XY_train, XY_Val =  train_test_split(XY_Data, test_size=0.3, random_state=228)
X_train, y_train = build_x_y_data(XY_train)    
X_val, y_val = build_x_y_data(XY_Val)

In [40]:
X_train.describe()

,atom_2,atom_3,atom_4,atom_5,atom_6,r_x_1,r_x_2,r_x_3,r_x_4,r_x_5,...,d_4_2,d_4_3,d_5_0,d_5_1,d_5_2,d_5_3,d_6_0,d_6_1,d_6_2,d_6_3
count,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,...,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000
mean,6.042530,3.134420,4.157781,4.955673,4.757038,-1.067688,0.212587,-0.282790,-0.479255,-0.770634,...,1.658874,2.392976,2.669473,1.935284,2.082391,2.431792,2.963041,2.069803,2.405976,2.647346
std,0.249587,2.569876,2.553271,2.140455,2.403251,0.034179,0.036439,0.640091,0.873691,0.887282,...,0.520463,0.597216,0.440065,0.543723,0.445237,0.783877,0.502180,0.614595,0.403336,0.774146
min,6.000000,1.000000,1.000000,1.000000,1.000000,-1.177559,0.084758,-2.000683,-2.490387,-3.436701,...,1.078170,1.078723,1.762759,1.005555,1.083384,1.006182,1.785467,1.005191,1.088632,1.004500
25%,6.000000,1.000000,1.000000,6.000000,1.000000,-1.088135,0.209613,-0.959353,-1.272946,-1.464971,...,1.098597,2.027695,2.267415,1.452749,1.550527,2.045917,2.541570,1.469537,2.223422,2.156590
50%,6.000000,1.000000,6.000000,6.000000,6.000000,-1.067222,0.220387,0.041553,-0.108669,-1.071961,...,1.527560,2.234305,2.656921,2.118483,2.209579,2.408760,3.015891,2.149516,2.458239,2.622725
75%,6.000000,6.000000,6.000000,6.000000,6.000000,-1.044226,0.229657,0.295155,0.300863,0.097556,...,2.145198,2.925505,3.002592,2.366688,2.440380,3.147599,3.325073,2.461068,2.617726,3.254888
max,8.000000,8.000000,8.000000,8.000000,9.000000,-0.916332,0.347900,0.590589,0.633578,2.316046,...,3.834378,3.877147,4.417424,3.746297,3.727495,4.339425,4.634705,4.104904,4.160572,4.676648


In [41]:
# Standard Scaler from sklearn does seem to work better here than other Scalers
scaler = StandardScaler()
train_input = scaler.fit_transform(X_train)   
cv_input =  scaler.transform(X_val)
train_target = y_train
cv_target = y_val

#train_input.describe()
#input_data=StandardScaler().fit_transform(df_train_.loc[:,input_features])
#target_data=df_train_.loc[:,"scalar_coupling_constant"].values

    


In [42]:
X_train.describe()

,atom_2,atom_3,atom_4,atom_5,atom_6,r_x_1,r_x_2,r_x_3,r_x_4,r_x_5,...,d_4_2,d_4_3,d_5_0,d_5_1,d_5_2,d_5_3,d_6_0,d_6_1,d_6_2,d_6_3
count,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,...,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000,8347.000000
mean,6.042530,3.134420,4.157781,4.955673,4.757038,-1.067688,0.212587,-0.282790,-0.479255,-0.770634,...,1.658874,2.392976,2.669473,1.935284,2.082391,2.431792,2.963041,2.069803,2.405976,2.647346
std,0.249587,2.569876,2.553271,2.140455,2.403251,0.034179,0.036439,0.640091,0.873691,0.887282,...,0.520463,0.597216,0.440065,0.543723,0.445237,0.783877,0.502180,0.614595,0.403336,0.774146
min,6.000000,1.000000,1.000000,1.000000,1.000000,-1.177559,0.084758,-2.000683,-2.490387,-3.436701,...,1.078170,1.078723,1.762759,1.005555,1.083384,1.006182,1.785467,1.005191,1.088632,1.004500
25%,6.000000,1.000000,1.000000,6.000000,1.000000,-1.088135,0.209613,-0.959353,-1.272946,-1.464971,...,1.098597,2.027695,2.267415,1.452749,1.550527,2.045917,2.541570,1.469537,2.223422,2.156590
50%,6.000000,1.000000,6.000000,6.000000,6.000000,-1.067222,0.220387,0.041553,-0.108669,-1.071961,...,1.527560,2.234305,2.656921,2.118483,2.209579,2.408760,3.015891,2.149516,2.458239,2.622725
75%,6.000000,6.000000,6.000000,6.000000,6.000000,-1.044226,0.229657,0.295155,0.300863,0.097556,...,2.145198,2.925505,3.002592,2.366688,2.440380,3.147599,3.325073,2.461068,2.617726,3.254888
max,8.000000,8.000000,8.000000,8.000000,9.000000,-0.916332,0.347900,0.590589,0.633578,2.316046,...,3.834378,3.877147,4.417424,3.746297,3.727495,4.339425,4.634705,4.104904,4.160572,4.676648


In [43]:
def create_nn_model(input_shape):
    inp = Input(shape=(input_shape,))
    x = Dense(2024, activation="relu")(inp)
    x = BatchNormalization()(x)
    #x = Dropout(0.4)(x)
    x = Dense(2024, activation="relu")(x)
    x = BatchNormalization()(x)
    out = Dense(1, activation="linear")(x)  
    model = Model(inputs=inp, outputs=[out])
    return model

In [44]:
# Set up GPU preferences
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2} ) 
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = tf.Session(config=config) 
K.set_session(sess)

In [45]:
# Build the Neural Net
mol_type = coupling_type
model_name_wrt = ('../input_added/NN/molecule_model_%s.hdf5' % coupling_type)
cv_score=[]
cv_score_total=0
epoch_n = 500 #1000
verbose = 1
batch_size = 2048
    
nn_model=create_nn_model(train_input.shape[1])
retrain = True
if not retrain:
    nn_model = load_model(model_name_wrt)
#rmropt = RMSprop(lr=0.0001)
nn_model.compile(loss='mae', optimizer=Adam())#, metrics=[auc])  
 # Callback for Early Stopping... May want to raise the min_delta for small numbers of epochs
es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=40,verbose=1, mode='auto', restore_best_weights=True)
# Callback for Reducing the Learning Rate... when the monitor levels out for 'patience' epochs, then the LR is reduced
rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=20, min_lr=1e-7, mode='auto', verbose=1)
# Save the best value of the model for future use
sv_mod = callbacks.ModelCheckpoint(model_name_wrt, monitor='val_loss', save_best_only=True, period=10)
history = nn_model.fit(train_input,[train_target], 
        validation_data=(cv_input,[cv_target]), 
        callbacks=[es, rlr, sv_mod], epochs=epoch_n, batch_size=batch_size, verbose=verbose)

Train on 8347 samples, validate on 3578 samples
Epoch 1/500
8347/8347 [==============================] - 2s 280us/step - loss: 7.2105 - val_loss: 4.5420
Epoch 2/500
8347/8347 [==============================] - 1s 127us/step - loss: 3.5293 - val_loss: 3.9757
Epoch 3/500
8347/8347 [==============================] - 1s 123us/step - loss: 2.9660 - val_loss: 2.5399
Epoch 4/500
8347/8347 [==============================] - 1s 125us/step - loss: 2.5006 - val_loss: 2.3127
Epoch 5/500
8347/8347 [==============================] - 1s 125us/step - loss: 2.2750 - val_loss: 2.2773
Epoch 6/500
8347/8347 [==============================] - 1s 121us/step - loss: 2.0176 - val_loss: 2.0429
Epoch 7/500
8347/8347 [==============================] - 1s 123us/step - loss: 1.7526 - val_loss: 1.6210
Epoch 8/500
8347/8347 [==============================] - 1s 123us/step - loss: 1.4688 - val_loss: 1.4411
Epoch 9/500
8347/8347 [==============================] - 1s 125us/step - loss: 1.1913 - val_loss: 1.3949
Epoch 1

Epoch 78/500
8347/8347 [==============================] - 1s 123us/step - loss: 0.4002 - val_loss: 0.6263
Epoch 79/500
8347/8347 [==============================] - 1s 124us/step - loss: 0.4169 - val_loss: 0.5859
Epoch 80/500
8347/8347 [==============================] - 1s 123us/step - loss: 0.4211 - val_loss: 0.5381
Epoch 81/500
8347/8347 [==============================] - 1s 126us/step - loss: 0.4200 - val_loss: 0.6744
Epoch 82/500
8347/8347 [==============================] - 1s 126us/step - loss: 0.4188 - val_loss: 0.5950
Epoch 83/500
8347/8347 [==============================] - 1s 122us/step - loss: 0.4309 - val_loss: 0.5999
Epoch 84/500
8347/8347 [==============================] - 1s 122us/step - loss: 0.4110 - val_loss: 0.5383
Epoch 85/500
8347/8347 [==============================] - 1s 122us/step - loss: 0.3979 - val_loss: 0.6313
Epoch 86/500
8347/8347 [==============================] - 1s 123us/step - loss: 0.4132 - val_loss: 0.5417
Epoch 87/500
8347/8347 [======================

Epoch 155/500
8347/8347 [==============================] - 1s 128us/step - loss: 0.3354 - val_loss: 0.4371
Epoch 156/500
8347/8347 [==============================] - 1s 126us/step - loss: 0.3192 - val_loss: 0.4109
Epoch 157/500
8347/8347 [==============================] - 1s 125us/step - loss: 0.3389 - val_loss: 0.3882
Epoch 158/500
8347/8347 [==============================] - 1s 128us/step - loss: 0.3193 - val_loss: 0.5254
Epoch 159/500
8347/8347 [==============================] - 1s 128us/step - loss: 0.3971 - val_loss: 0.4458
Epoch 160/500
8347/8347 [==============================] - 1s 127us/step - loss: 0.4007 - val_loss: 0.4856
Epoch 161/500
8347/8347 [==============================] - 1s 124us/step - loss: 0.3860 - val_loss: 0.5455
Epoch 162/500
8347/8347 [==============================] - 1s 127us/step - loss: 0.3822 - val_loss: 0.4895
Epoch 163/500
8347/8347 [==============================] - 1s 126us/step - loss: 0.3501 - val_loss: 0.4796
Epoch 164/500
8347/8347 [============


Epoch 00230: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 231/500
8347/8347 [==============================] - 1s 125us/step - loss: 0.2244 - val_loss: 0.3088
Epoch 232/500
8347/8347 [==============================] - 1s 126us/step - loss: 0.2237 - val_loss: 0.3058
Epoch 233/500
8347/8347 [==============================] - 1s 126us/step - loss: 0.2169 - val_loss: 0.3009
Epoch 234/500
8347/8347 [==============================] - 1s 125us/step - loss: 0.2010 - val_loss: 0.2970
Epoch 235/500
8347/8347 [==============================] - 1s 124us/step - loss: 0.2090 - val_loss: 0.2982
Epoch 236/500
8347/8347 [==============================] - 1s 126us/step - loss: 0.2127 - val_loss: 0.2977
Epoch 237/500
8347/8347 [==============================] - 1s 127us/step - loss: 0.1986 - val_loss: 0.2985
Epoch 238/500
8347/8347 [==============================] - 1s 127us/step - loss: 0.2098 - val_loss: 0.2994
Epoch 239/500
8347/8347 [==============================] - 1s 

Epoch 307/500
8347/8347 [==============================] - 1s 128us/step - loss: 0.2161 - val_loss: 0.2916

Epoch 00307: ReduceLROnPlateau reducing learning rate to 8.000000525498762e-06.
Epoch 308/500
8347/8347 [==============================] - 1s 125us/step - loss: 0.2012 - val_loss: 0.2913
Epoch 309/500
8347/8347 [==============================] - 1s 130us/step - loss: 0.1937 - val_loss: 0.2905
Epoch 310/500
8347/8347 [==============================] - 1s 125us/step - loss: 0.2031 - val_loss: 0.2908
Epoch 311/500
8347/8347 [==============================] - 1s 128us/step - loss: 0.1937 - val_loss: 0.2905
Epoch 312/500
8347/8347 [==============================] - 1s 128us/step - loss: 0.1905 - val_loss: 0.2904
Epoch 313/500
8347/8347 [==============================] - 1s 127us/step - loss: 0.2011 - val_loss: 0.2905
Epoch 314/500
8347/8347 [==============================] - 1s 127us/step - loss: 0.2091 - val_loss: 0.2907
Epoch 315/500
8347/8347 [==============================] - 1s 1


Epoch 00382: ReduceLROnPlateau reducing learning rate to 1.6000001778593287e-06.
Epoch 383/500
8347/8347 [==============================] - 1s 126us/step - loss: 0.2193 - val_loss: 0.2891
Epoch 384/500
8347/8347 [==============================] - 1s 127us/step - loss: 0.1926 - val_loss: 0.2889
Epoch 385/500
8347/8347 [==============================] - 1s 130us/step - loss: 0.1926 - val_loss: 0.2888
Epoch 386/500
8347/8347 [==============================] - 1s 130us/step - loss: 0.1928 - val_loss: 0.2889
Epoch 387/500
8347/8347 [==============================] - 1s 128us/step - loss: 0.1944 - val_loss: 0.2888
Epoch 388/500
8347/8347 [==============================] - 1s 127us/step - loss: 0.1869 - val_loss: 0.2889
Epoch 389/500
8347/8347 [==============================] - 1s 125us/step - loss: 0.1965 - val_loss: 0.2890
Epoch 390/500
8347/8347 [==============================] - 1s 129us/step - loss: 0.1935 - val_loss: 0.2890
Epoch 391/500
8347/8347 [==============================] - 1s 

In [46]:
import matplotlib.ticker as tic

def plot_history(history, label):
    fig1, ax1 = plt.subplots()
    ax1.plot(history.history['loss'])
    ax1.plot(history.history['val_loss'])
    #ax1.title('Loss for %s' % label)
    #ax1.ylabel('Loss')
    #ax1.xlabel('Epoch')
   # ax1.set_yscale('log')
    ax1.set_yticks([0.1, 0.2, 0.3, 0.4, 0.6])
    plt.ylim([0.05, 0.2])
    ax1.get_yaxis().set_major_formatter(tic.ScalarFormatter())
    ax1.legend(['Train','Validation'], loc='upper left')
    plt.show()

In [ ]:
cv_predict=nn_model.predict(cv_input)
plot_history(history, mol_type)
accuracy=np.mean(np.abs(cv_target-cv_predict[:,0]))
print(np.log(accuracy))
cv_score.append(np.log(accuracy))

In [ ]:
accuracy